# DeutscheWelle Data Engineering Challenge

9.05.2023


# Task 1

## Connect via Gmail API



In [ ]:
## Script that dowloads emails from gmail inbox

## from https://developers.google.com/gmail/api/reference/rest/v1/users.messages.attachments/get 
# # Method to download email attachments from gmail api
##
# userId dwdechallenge@gmail.com / me (authenticated user)
# messageId 
# id  2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv 
#GET https://gmail.googleapis.com/gmail/v1/users/{userId}/messages/{messageId}/attachments/{id}

In [1]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
import base64
import os
import time
import sys 

# Import functions from source folder
sys.path.append('../src/') 
from api_functions import *


In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
creds = Credentials.from_authorized_user_file('../token.json', SCOPES)
service = build('gmail', 'v1', credentials=creds)

## search for emails with 'report' in the attachment name
query_string='filename:report*csv' #find all the email with attachment in csv format containing 'report' in their name
# other possible queries
#query_string='report' # all email with report in subject/body text
#query_string='has:attachment' # all email with attachments
email_messages = search_email(service, query_string,['INBOX'])
try:
    path = os.path.join('../', 'data/downloaded')
    os.mkdir(path)
except OSError as error:
    print(error) 
save_location='../data/downloaded'
## download attachments for found emails 

for email_message in email_messages:
    #print(email_message['id'])
    #187ffb2fbf3c1bc5
    messageDetail = get_message_detail(service,email_message['id'],msg_format='full',metadata_headers=['parts'])
    #print(messageDetail)
    messageDetailPayload = messageDetail.get('payload')

    if 'parts' in messageDetailPayload:
        for msgPayload in messageDetailPayload['parts']:
            file_name = msgPayload['filename']
            body=msgPayload['body']

            if 'attachmentId' in body:
                attachment_id = body['attachmentId']
                attachment_content= get_file_data(service,email_message['id'],attachment_id,file_name,save_location)
                print(file_name)
                
                #with open(os.path.join(save_location + '/' 'Downloaded_files' + '/' + file_name),'wb') as _f:
                with open(os.path.join(save_location + '/' + file_name),'wb') as _f:
                    _f.write(attachment_content)
                    print(f'File {file_name} is saved at {save_location}')
    time.sleep(0.5)
    #break

[Errno 17] File exists: '../data/downloaded'
2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv
File 2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv is saved at ../data/downloaded
2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv
File 2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv is saved at ../data/downloaded
2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv
File 2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv is saved at ../data/downloaded
2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv
File 2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv is saved at ../data/downloaded
2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv
File 2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2

## Read the data from the files and upload them to the postgres database.

In [2]:
import pandas as pd
import psycopg2
import os
import glob
import sys

# Import functions from source folder
sys.path.append('../src/') 
from postgres_functions import *

In [3]:
#downloaded report .csv files

daily_repo_filepath='../data/downloaded/' # downloaded files via API
files=glob.glob(os.path.join(daily_repo_filepath,"*.csv"))
files

['../data/downloaded/2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv',
 '../data/downloaded/2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv',
 '../data/downloaded/2022-09-21-23-04-26-EDT-Historical-Report-ENTRFacebook-2022-06-22--2022-09-22.csv',
 '../data/downloaded/2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv',
 '../data/downloaded/2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv',
 '../data/downloaded/2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv',
 '../data/downloaded/2022-09-23-23-04-15-EDT-Historical-Report-ENTRFacebook-2022-06-24--2022-09-24.csv']

In [4]:
#check the data with pandas
data = pd.read_csv(files[0])
data.head()

,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Message,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Overperforming Score
0,Onet Styl Życia,onetstylzycia,372341662829366,NEWS_SITE,PL,Najbardziej stylowy serwis w Polsce!,2012-06-05 16:57:21,558890,563792,2022-09-18 21:00:26 EDT,...,Wyrwał: w rękach Rosjan jest ok. 17 proc. tery...,https://www.facebook.com/onetstylzycia/videos/...,NaN,NaN,"""To jest przełom w wojnie w Ukrainie, zdobywam...",NaN,NaN,NaN,NaN,-0.89
1,DW News,deutschewellenews,100064359716848,NEWS_SITE,DE,Follow us for trustworthy news and analysis fr...,2008-08-26 12:26:42,2664421,3150873,2022-09-18 20:12:16 EDT,...,Every year 400 million tonnes of plastic are p...,https://www.facebook.com/deutschewellenews/vid...,NaN,NaN,Highlighting the plastic waste problem,NaN,NaN,NaN,NaN,-3.29
2,DW Euromaxx,dw.euromaxx,211745678846355,BROADCASTING_MEDIA_PRODUCTION,DE,Euromaxx is made for those who love European #...,2011-05-17 14:08:01,501640,756194,2022-09-18 19:15:16 EDT,...,"🥚🥔 In Spain, they're eaten for breakfast, lunc...",https://www.facebook.com/dw.euromaxx/videos/47...,NaN,NaN,The Secrets Behind the Traditional Spanish Tor...,NaN,NaN,NaN,NaN,1.01
3,Onet Wiadomości,OnetWiadomosci,56024223012,NEWS_SITE,PL,"Jeśli macie jakieś uwagi, chcecie pokazać, jak...",2009-01-13 10:54:16,533859,625613,2022-09-18 19:00:20 EDT,...,"""Staram się, aby to nie była zwykła relacja z ...",https://www.facebook.com/OnetWiadomosci/videos...,NaN,NaN,"""Robię filmiki komediowe, podróżnicze, ciekawo...",NaN,NaN,NaN,NaN,-5.17
4,Konbini news,konbininews,183081005507412,MEDIA_NEWS_COMPANY,FR,Society & Politics by Konbini,2017-01-30 10:03:14,722701,1546605,2022-09-18 16:05:01 EDT,...,"Algorithme, limites entre vie pro et perso, me...",https://buff.ly/3BHd46t,https://www.konbini.com/videos/enjoyphoenix-de...,NaN,EnjoyPhoenix dévoile les réalités du métier de...,"Algorithme, limites entre vie pro et perso, me...",NaN,NaN,NaN,-4.72


In [ ]:
# connect to postgres database
#parameters
param_dic = {
    "host"      : "localhost",
    "database"  : "postgres",
    "user"      : "postgres",
}

In [6]:
#create postgres database

# column names from pandas dataframe csv file
cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])

#name of a new postgres database 
new_table='table101'

#query=("DROP TABLE IF EXISTS %s;"%(new_table))
query=("CREATE TABLE IF NOT EXISTS %s(%s)" %(new_table,(cols))) 

conn = connect(param_dic)
execute_query(conn, query)
#create_postgres(data,table)

#name of the postgres database to upload data
table=new_table

#upload data from csv files to postreg database
[upload_csv_to_postgres(table,files) for file in files]

Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful


[None, None, None, None, None, None, None]

In [ ]:
conn = connect(param_dic)
execute_query(conn, 'SELECT Page_Name User_Name FROM new_table')

In [ ]:
##SQL script to process the data and push the data to GIT
## create tables 


# Task 2

#### Have a look at the dwh_dl_facebook_post_insights.csv. You will find there 3 rows for seperate posts. Import the csv to a database table

In [7]:
path_insights='/Users/anya/SCIENCE/DW_Challenge/data/external/Junior_DE_Task/'
db_insights=pd.read_csv(path_insights+'dwh_dl_facebook_post_insights.csv')

In [8]:
db_insights

,insights_created_date,post_activity,post_activity_by_action_type,post_activity_by_action_type_unique,post_activity_unique,post_clicks,post_clicks_by_type,post_clicks_by_type_unique,post_clicks_unique,post_engaged_fan,...,post_video_views_autoplayed,post_video_views_by_distribution_type,post_video_views_clicked_to_play,post_video_views_organic,post_video_views_organic_unique,post_video_views_paid,post_video_views_paid_unique,post_video_views_sound_on,post_video_views_unique,period
0,2019-09-29,67,"{""share"": 6, ""like"": 56, ""comment"": 5}","{""share"": 5, ""like"": 56, ""comment"": 5}",65,384,"{""video play"": 307, ""other clicks"": 71, ""link ...","{""video play"": 303, ""other clicks"": 60, ""link ...",355,31,...,8074,"{""page_owned"": 8264, ""shared"": 48}",238,218,204,8094,7284,1445,7487,lifetime
1,2019-09-29,65,"{""share"": 5, ""like"": 58, ""comment"": 2}","{""share"": 5, ""like"": 58, ""comment"": 2}",61,352,"{""video play"": 265, ""other clicks"": 85}","{""video play"": 265, ""other clicks"": 71}",323,34,...,4467,"{""page_owned"": 4544, ""shared"": 143}",220,269,245,4418,4142,1391,4399,lifetime
2,2019-09-29,93,"{""share"": 9, ""like"": 84}","{""share"": 8, ""like"": 82}",87,327,"{""video play"": 272, ""other clicks"": 53}","{""video play"": 259, ""other clicks"": 48}",303,38,...,5900,"{""shared"": 50, ""page_owned"": 6035}",185,140,116,5945,5445,955,5558,lifetime


In [9]:

table='insights_draft'
data=db_insights
files_ins=glob.glob(os.path.join(path_insights,"*.csv"))
#conn = connect(param_dic)
create_postgres(data,table)
upload_csv_to_postgres(table,files_ins)

NameError: name 'data' is not defined

In [ ]:
files_ins

### Please write a SQL script to unpack the json in column post_video_view_time_by_age_bucket_and_gender in a new table, while preserving the other columns.

In [ ]:
SELECT 
   id, 
   data::json->'name' as name,
   data::json->'author' ->> 'last_name' as author
FROM books;

In [ ]:
conn = connect(param_dic)

sel_column='post_video_view_time_by_age_bucket_and_gender'
table='insights_draft'
execute_query(conn,'SELECT %s FROM %s'%(sel_column,table))

In [ ]:
execute_query(conn,'SELECT %s FROM %s'%(sel_column,table))

In [ ]:
execute_query(conn, "SELECT %s  FROM %s, data::jsons->'U.18-24' as U.18-24, data::jsons->'U.25-34' as U.25-34"%(sel_column,table))

In [ ]:
execute_query(conn,'SELECT %s FROM %s, json_each(%s)'%(sel_column,table,sel_column))

In [ ]:
print(sel_column)
conn = connect(param_dic)
execute_query(conn, "SELECT %s json_data -> '%s' %s, FROM %s;" %(sel_column,sel_column,'foo',table))

In [ ]:
table

In [ ]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    SELECT json_data 
        post_video_view_time_by_age_bucket_and_gender -> "U.18-24" as U1
    FROM insights_draft;
    """)
conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    SELECT (post_video_view_time_by_age_bucket_and_gender) from insights_draft;
    """)
conn.commit()
conn.close()

In [ ]:
execute_query(conn,'select jsonb_object_keys(%s) from %s;'%(sel_column,table))

In [ ]:
table

In [ ]:
print(sel_column)
conn = connect(param_dic)
execute_query(conn, "SELECT %s -> 'U.18-24' AS U1 FROM %s" %(sel_column,table))

#SELECT post_video_view_time_by_age_bucket_and_gender FROM insights_draft;

#SELECT post_video_view_time_by_age_bucket_and_gender json_data ->> 'U.18-24' AS U1 FROM insights_draft; 

In [ ]:
execute_query(conn,"select 
    json_data #> 
       '{pizzas,1,additionalToppings}'  
        as additional_toppings_2nd_pizza
from test;")

### Write a script for the sums per post for each gender. Additionally create the sum for age 18-34 of all genders.

In [ ]:
# SELECT SUM(Quantity) FROM OrderDetails;
# add where statement WHERE Country='Mexico' WHERE Country='Mexico'
# SELECT * FROM Products WHERE Price BETWEEN 50 AND 60;